## Week 4 and 5 Capstone Project - The Battle of Neighborhoods

### For this project, I want to find out which city, New York City or Toronto, is more vegan friendly, so that I can recommend one over the other as a better travel destination for my vegan clients at my fictional travel agenecy business.


### NOTE: if there is a key error: ‘groups’ when running the code in the notebook, it might be because the number of calls I can make with my free foursquare account is limited and as a result the quota might have been exceeded. I tried running my code and I couldn't get the venue data and then plot the map data as a result after I reset it one too many times. Please look at my work closely as a result, and not just the output or lack of output. Thank You!


In [40]:
import pandas as pd 
import numpy as np 

from bs4 import BeautifulSoup
import requests 

import geocoder 
import folium
from geopy.geocoders import Nominatim

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.cm as cm
import matplotlib.colors as colors

### Firstly, we need to make a table for New York and Toronto, which contains the Neighborhoods of each city, the postal codes/zip codes, borough, and the Lat-Long coordinates for each neighborhood.



In [41]:
#New York

# #grab URL and store it in a URL variable
NURL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json"

print(NURL)

# #obtain raw html
nyr = requests.get(NURL).json()

nydata = nyr["features"]

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
nyneigh = pd.DataFrame(columns=column_names)    

for data in nydata:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyneigh = nyneigh.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

nyneigh.head()


https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [42]:
#toronto

#grab URL and store it in a URL variable
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#obtain raw html
r = requests.get(URL)
#parse into beautifulsoup 
soup = BeautifulSoup(r.content) 
#find required content
table = soup.findAll('table',attrs ={'rules':'all','cellspacing':0,}) 
#convert to text
a = table[0].text

"""
beautiful soup text has a lot of newlines. split by new line, and then run through
the rest of the code to remove the newline and replace it with an empty char
"""
b = a.split("\n")
for word in b:
    word.replace("\n", "")
    word.replace("\n", "")
    word.replace("\n", "")
#create a list
c = []
#now only add stuff that isn't an empty char or does not contain Not Assigned
for word in b:
    if "Not assigned" not in word and word != "":
        c.append(word)
#create a new table to store results
table_contents=[]

for word in c:
    #dict to store values for each neighborhood
    cell = {}
    #Since each postal code is 3 digits, go till the third digit and set it as the postal code value
    cell['Postal Code'] = word[:3]
    #set a temp variable equal to the rest of the word, which has the borough and the neighborhood
    temp = word.replace(word[:3], '')
    #split by the parenthesis, and grab the first half which is the borough, set it to the borough of the cell
    cell['Borough'] = temp.split('(')[0]
    #same logic for the neighborhood, just strip all the uneeded characters
    cell['Neighborhood'] = (((((temp).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    #append the dict to the table
    table_contents.append(cell)
#convert to trawneigh
trawneigh=pd.DataFrame(table_contents)
#handle special cases
trawneigh['Borough']=trawneigh['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

f = open("/Users/shreyasiyengar/VSC/Coursera_Capstone/Geospatial_Coordinates.csv", "r")
trawnocoords = pd.read_csv("/Users/shreyasiyengar/VSC/Coursera_Capstone/Geospatial_Coordinates.csv",",",header = 0)



trawneigh.head()
    
trawneigh = pd.merge(trawneigh, trawnocoords[['Postal Code', 'Latitude', 'Longitude']], )
trawneigh.head()
    

/Users/shreyasiyengar/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Now that we have the neighborhood tables for NYC and Toronto, we need to grab the foursquare venue tables for each city.

In [43]:
# Foursquare Credential Prep
CLIENT_ID = 'KOGTV4EAQCBU054ZKLCRMAINCATFAHUJWLHPTAUZ4A3N33SG' # your Foursquare ID
CLIENT_SECRET = 'R0X1HHFDDDIQF5EHZHICRSD1IX2J4BWNEWP4VF3XFC43EYDP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: KOGTV4EAQCBU054ZKLCRMAINCATFAHUJWLHPTAUZ4A3N33SG
CLIENT_SECRET:R0X1HHFDDDIQF5EHZHICRSD1IX2J4BWNEWP4VF3XFC43EYDP


#### The following code, following the New York Lab, gets the nearby venues for a neighborhood using the lat-long, we then use this function to grab the nearby venues for each neighborhood in our dataset.

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

    

In [46]:
trawno_venues = getNearbyVenues(names=trawneigh['Neighborhood'],
                                   latitudes= trawneigh['Latitude'],
                                   longitudes=trawneigh['Longitude']
                                  )
nyc_venues = getNearbyVenues(names=nyneigh['Neighborhood'],
                                   latitudes= nyneigh['Latitude'],
                                   longitudes=nyneigh['Longitude']
                                  )                                 

Parkwoods


KeyError: 'groups'

NOTE: if there is a key error: ‘groups’ when running the code in the notebook, it might be because the number of calls I can make with my free foursquare account is limited and as a result the quota might have been exceeded.


In [ ]:
trawno_venues.head()

In [ ]:
nyc_venues.head()

In [ ]:
trawveg = trawno_venues.loc[trawno_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
trawveg.head()

In [ ]:
nyveg = nyc_venues.loc[nyc_venues['Venue Category'] == 'Vegetarian / Vegan Restaurant']
nyveg.head()

In [ ]:
taddress = 'Toronto, ON'

geolocator = Nominatim(user_agent="trawno_explorer")
tlocation = geolocator.geocode(taddress)
tlatitude = tlocation.latitude
tlongitude = tlocation.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(tlatitude, tlongitude))


In [ ]:
naddress = 'New York City, NY'

geolocator = Nominatim(user_agent="nyc_explorer")
nlocation = geolocator.geocode(naddress)
nlatitude = nlocation.latitude
nlongitude = nlocation.longitude
print('The geograpical coordinates of New York City are {}, {}.'.format(nlatitude, nlongitude))


In [ ]:
map_trawno = folium.Map(location=[tlatitude, tlongitude], zoom_start=10)

# add markers to map
for lat, lng, venue, neighborhood in zip(trawveg['Neighborhood Latitude'], trawveg['Neighborhood Longitude'], trawveg['Venue'], trawveg['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ffffff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trawno)  
    
map_trawno

In [ ]:
 map_ny = folium.Map(location=[nlatitude, nlongitude], zoom_start=10)

# add markers to map
for lat, lng, venue, neighborhood in zip(nyveg['Neighborhood Latitude'], nyveg['Neighborhood Longitude'], nyveg['Venue'], nyveg['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#ffffff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny